## Please input your directory for the top level folder
folder name : SUBMISSION MODEL

In [1]:
dir_ = 'E:/Seminararbeit/Code/A1/' # input only here

#### setting other directory

In [2]:
raw_data_dir = dir_+'2. data/'
processed_data_dir = dir_+'2. data/processed/'
log_dir = dir_+'4. logs/'
model_dir = dir_+'5. models/'
submission_dir = dir_+'6. submissions/'

In [3]:
####################################################################################
####################### 1-2. recursive model by store & cat ########################
####################################################################################

In [4]:
ver, KKK = 'priv', 1

In [5]:
STORES = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']
CATS = ['HOBBIES','HOUSEHOLD', 'FOODS']

In [6]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

# custom imports
from multiprocessing import Pool

warnings.filterwarnings('ignore')

In [7]:
########################### Helpers
#################################################################################
## Seeder
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

    
## Multiprocess Runs
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

In [8]:
########################### Helper to load data by store ID
#################################################################################
# Read data
def get_data_by_store(store, dept):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    df = df[df['d']>=START_TRAIN]
    
    df = df[(df['store_id']==store) & (df['cat_id']==dept)]

    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2
    
    df = pd.concat([df, df3], axis=1)
    del df3
    
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    df = df.reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES:
        for state_id in CATS:
            temp_df = pd.read_pickle(processed_data_dir+'test_'+store_id+'_'+state_id+'.pkl')
            temp_df['store_id'] = store_id
            temp_df['cat_id'] = state_id
            base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


########################### Helper to make dynamic rolling lags
#################################################################################
def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]


def make_lag_roll(LAG_DAY):
    shift_day = LAG_DAY[0]
    roll_wind = LAG_DAY[1]
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    return lag_df[[col_name]]

In [9]:
########################### Model params
#################################################################################
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.015,
                    'num_leaves': 2**8-1,
                    'min_data_in_leaf': 2**8-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 3000,
                    'boost_from_average': False,
                    'verbose': -1
                } 

In [10]:
########################### Vars
#################################################################################
VER = 1                          
SEED = 42                        
seed_everything(SEED)            
lgb_params['seed'] = SEED        
N_CORES = psutil.cpu_count()     


#LIMITS and const
TARGET      = 'sales'            
START_TRAIN = 700                
END_TRAIN   = 1941 - 28*KKK      
P_HORIZON   = 28                 
USE_AUX     = False             

remove_features = ['id','cat_id', 'state_id','store_id',
                   'date','wm_yr_wk','d',TARGET]
mean_features   = ['enc_store_id_dept_id_mean','enc_store_id_dept_id_std',
                   'enc_item_id_store_id_mean','enc_item_id_store_id_std'] 

ORIGINAL = raw_data_dir
BASE     = processed_data_dir+'processedgrid_part_1.pkl'
PRICE    = processed_data_dir+'processedgrid_part_2.pkl'
CALENDAR = processed_data_dir+'processedgrid_part_3.pkl'
LAGS     = processed_data_dir+'processedlags_df_28.pkl'
MEAN_ENC = processed_data_dir+'processedmean_encoding_df.pkl'


#SPLITS for lags creation
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,30,60]:
        ROLS_SPLIT.append([i,j])

In [11]:
_, MODEL_FEATURES = get_data_by_store(STORES[-1], CATS[-1])
del _; gc.collect()

0

In [12]:
########################### Predict
#################################################################################

all_preds = pd.DataFrame()

# Join back the Test dataset with 
# a small part of the training data 
# to make recursive features
base_test = get_base_test()

main_time = time.time()

for PREDICT_DAY in range(1,29):    
    print('Predict | Day:', PREDICT_DAY)
    start_time = time.time()

    grid_df = base_test.copy()
    
    # slow for loop version
    temp = []
    for a in ROLS_SPLIT:
        temp.append(make_lag_roll(a))
    temp = pd.concat(temp, axis=1)
    grid_df = pd.concat([grid_df, temp], axis=1)
    del temp; gc.collect()
    ###
    
    # fast multiprocessing version
#     grid_df = pd.concat([grid_df, df_parallelize_run(make_lag_roll, ROLS_SPLIT)], axis=1)
    ###
        
    for store_id in STORES:
        for state_id in CATS:
        
            model_path = model_dir+'lgb_model_'+store_id+'_'+state_id+'_v'+str(VER)+'.bin'
            if USE_AUX:
                model_path = AUX_MODELS + model_path

            estimator = pickle.load(open(model_path, 'rb'))

            day_mask = base_test['d']==(END_TRAIN+PREDICT_DAY)
            store_mask = base_test['store_id']==store_id
            state_mask = base_test['cat_id']==state_id

            mask = (day_mask)&(store_mask)&(state_mask)
            base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])
    
    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id','F'+str(PREDICT_DAY)]
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()
        
    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    del temp_df
    
all_preds = all_preds.reset_index(drop=True)
all_preds

Predict | Day: 1
##########  2.11 min round |  2.11 min total |  37362.33 day sales |
Predict | Day: 2
##########  1.76 min round |  3.87 min total |  35820.30 day sales |
Predict | Day: 3
##########  1.75 min round |  5.62 min total |  35287.56 day sales |
Predict | Day: 4
##########  1.75 min round |  7.38 min total |  35864.35 day sales |
Predict | Day: 5
##########  1.76 min round |  9.13 min total |  42346.81 day sales |
Predict | Day: 6
##########  1.76 min round |  10.89 min total |  51529.00 day sales |
Predict | Day: 7
##########  1.76 min round |  12.65 min total |  54663.97 day sales |
Predict | Day: 8
##########  1.76 min round |  14.41 min total |  44911.33 day sales |
Predict | Day: 9
##########  1.76 min round |  16.16 min total |  45049.70 day sales |
Predict | Day: 10
##########  1.76 min round |  17.92 min total |  39771.12 day sales |
Predict | Day: 11
##########  1.76 min round |  19.68 min total |  41129.39 day sales |
Predict | Day: 12
##########  1.75 min round |

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_evaluation,0.870932,0.764649,0.705301,0.771919,0.892273,1.495131,2.305302,0.844798,0.808506,...,0.890030,1.335815,0.969617,0.856054,0.736404,0.755669,0.851076,1.120754,1.275398,1.115024
1,HOBBIES_1_002_CA_1_evaluation,0.156725,0.179041,0.139369,0.157679,0.171849,0.218373,0.205946,0.198877,0.223985,...,0.225279,0.276965,0.301717,0.216404,0.215209,0.225649,0.250528,0.243223,0.337103,0.306588
2,HOBBIES_1_003_CA_1_evaluation,0.405382,0.400993,0.511306,0.489348,0.676125,1.198966,0.786767,0.422763,0.426771,...,0.662712,0.847849,0.868454,0.489339,0.482109,0.502099,0.549881,0.810057,0.972190,0.877789
3,HOBBIES_1_004_CA_1_evaluation,1.444013,1.247191,1.457564,1.482993,1.992003,1.999145,5.015667,1.795505,1.303647,...,1.716196,2.209101,3.038259,1.705341,1.328901,1.249291,1.264975,1.533475,3.448009,3.744222
4,HOBBIES_1_005_CA_1_evaluation,0.930047,0.880204,0.932612,1.169816,1.061837,1.087882,1.938193,1.017873,1.115508,...,1.218561,1.698868,1.563112,1.012973,0.957349,0.974835,0.972285,1.238403,1.543711,1.607362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,0.378075,0.363264,0.378961,0.369856,0.456488,0.442834,0.514963,0.537534,0.563089,...,0.583207,0.697551,0.821740,0.564087,0.484115,0.458311,0.453009,0.509889,0.617922,0.704304
30486,FOODS_3_824_WI_3_evaluation,0.319542,0.302139,0.293962,0.284748,0.287117,0.285719,0.317155,0.419328,0.412290,...,0.263523,0.345654,0.418916,0.294155,0.261761,0.248264,0.228003,0.217350,0.293158,0.316587
30487,FOODS_3_825_WI_3_evaluation,0.601561,0.515258,0.497002,0.494971,0.615638,0.819971,0.934800,1.174792,1.147953,...,1.000693,1.335199,1.609247,1.086334,0.767945,0.692199,0.640633,0.738843,0.847282,0.820046
30488,FOODS_3_826_WI_3_evaluation,1.090637,1.115649,0.936578,1.003429,1.118560,1.358563,1.102913,1.292304,1.319514,...,0.990432,1.401226,1.442792,1.168127,1.075675,0.947878,1.005170,1.032993,1.286514,1.408132


In [13]:
########################### Export
#################################################################################
submission = pd.read_csv(ORIGINAL+'sample_submission.csv')[['id']]
submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
submission.to_csv(submission_dir+'before_ensemble/submission_kaggle_recursive_store_cat.csv', index=False)